In [1]:
import numpy as np
import pandas as pd
import folium
from branca.colormap import linear

station_stats = pd.read_csv('./data/station_stats.csv')
START_YEAR = 1980
END_YEAR = 2020
num_years = END_YEAR - START_YEAR
station_stats['pct_precip_change'] = (100 * station_stats.slope_total_precip_pct).round(2)
station_stats.head()

,station_id,slope_max_temp,slope_min_temp,slope_total_precip_pct,missing_precip_freq,years_zero_reported_precip,municipality,lat,lon,region,sub-region,country,elevation,pct_precip_change
0,ASN00009021,0.040568,0.001091,-0.004958,0.000000,0,Perth,-32.0,116.0,Oceania,Australia and New Zealand,Australia,15.0,-0.50
1,ASN00023034,0.026023,0.025608,-0.003983,0.000067,0,Adelaide,-35.0,139.0,Oceania,Australia and New Zealand,Australia,2.0,-0.40
2,ASN00040043,0.045486,0.026000,-0.014759,0.042978,0,Brisbane,-27.0,153.0,Oceania,Australia and New Zealand,Australia,100.0,-1.48
3,ASN00066037,0.043564,0.040078,-0.002078,0.000401,0,Sydney,-34.0,151.0,Oceania,Australia and New Zealand,Australia,6.0,-0.21
4,ASN00086282,0.038020,0.017193,-0.000682,0.000000,0,Melbourne,-38.0,145.0,Oceania,Australia and New Zealand,Australia,113.0,-0.07


In [2]:
raw_data = pd.read_parquet('./data/weather_1970_to_2020.parquet')
raw_data['date'] = pd.to_datetime(raw_data.date)
raw_data['year'] = raw_data.date.dt.year
annual_stats = raw_data.fillna(0).groupby(['station_id', 'year']).agg(
                        {'max_temp_c': ['mean'],
                         'min_temp_c': ['mean'],
                         'precip_mm': ['sum']}
).reset_index()
annual_stats.columns = ['station_id', 'year', 'max_temp_c', 'min_temp_c', 'precip_mm']

station_stats = pd.read_csv('./data/station_stats.csv')
station_stats['pct_precip_change'] = (100 * station_stats.slope_total_precip_pct.round(3))
station_stats['slope_max_temp'] = station_stats['slope_max_temp'].round(2)
station_stats['slope_min_temp'] = station_stats['slope_min_temp'].round(2)

annual_stats

,station_id,year,max_temp_c,min_temp_c,precip_mm
0,ASN00009021,1970,23.972877,12.514521,8475.0
1,ASN00009021,1971,23.017260,11.574795,7760.0
2,ASN00009021,1972,25.535246,12.084426,5768.0
3,ASN00009021,1973,23.798630,11.833151,9146.0
4,ASN00009021,1974,24.190685,11.918356,8758.0
...,...,...,...,...,...
8895,USW00094931,2015,11.354270,-2.009917,6283.0
8896,USW00094931,2016,10.292350,-2.122678,7284.0
8897,USW00094931,2017,9.312637,-3.417857,7208.0
8898,USW00094931,2018,8.866484,-4.137637,4274.0


In [3]:
# alt.themes.enable('fivethirtyeight')

In [4]:
import altair as alt
city = station_stats.iloc[5]
annual_this_city = annual_stats.loc[annual_stats.station_id == city.station_id]
scatter = alt.Chart(annual_this_city, title=city.municipality).mark_point().encode(
    alt.X('year', scale=alt.Scale(zero=False), axis=alt.Axis(format="d")),
    alt.Y('max_temp_c', scale=alt.Scale(zero=False))
)
full_chart = scatter + scatter.transform_regression('year', 'max_temp_c').mark_line()
full_chart

alt.LayerChart(...)

In [22]:
station_stats

,station_id,slope_max_temp,slope_min_temp,slope_total_precip_pct,missing_precip_freq,years_zero_reported_precip,municipality,lat,lon,sub-region,country,elevation,pct_precip_change
region,,,,,,,,,,,,,
Americas,150,150,150,150,150,150,150,150,150,150,150,150,150
Asia,1,1,1,1,1,1,1,1,1,1,1,1,1
Europe,59,59,59,59,59,59,59,59,59,59,59,59,59
Oceania,6,6,6,6,6,6,6,6,6,6,6,6,6
